In [13]:
import pandas as pd
import requests
import datetime

import env

In [14]:
search_base_url = 'https://www.googleapis.com/youtube/v3/search'
videos_base_url = 'https://www.googleapis.com/youtube/v3/videos'

search_params = {
  'key': env.token,
  'part': 'snippet',
  'order': 'viewCount',
  'regionCode': 'KR',
  'type': 'video',
  'maxResults': 20,
  'fields': 'items(id/videoId, snippet(publishedAt,title))'
}
videos_params = {
  'key': env.token,
  'part': 'statistics',
  'fields': 'items/statistics'
}

search_keywords = ['넷플릭스', '명장면', '리뷰', '해석', '의미']
columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']

tokenLimit = len(env.token)-1

In [15]:
movieInfo = pd.read_excel('test-data-netflix.xlsx').loc[:,['movie_name', 'release']]
movieInfo.columns = ['영화명', '개봉일']
movieInfo = movieInfo.astype({'개봉일': 'str'})
movieNames = list(movieInfo.loc[:, '영화명'])
movieInfo

,영화명,개봉일
0,오징어 게임,2021-09-17
1,킹덤,2019-01-25
2,보건교사 안은영,2020-09-25
3,옥자,2017-05-19
4,승리호,2021-02-05


In [16]:
startIndex = 0
endIndex = len(movieNames)

def get_video_list():
    columns = ['영상 id', '영화 제목', '검색 키워드', '영상 제목', '조회수', '좋아요수', '댓글 수', '생성된 날짜']
    video_list = pd.DataFrame(columns=columns)
    tokenNum = 0

    for movieName in movieNames[startIndex:endIndex]:
        print(movieName, len(video_list))
        for keyword in search_keywords:
            # parameter 설정
            publishedAfter = list(movieInfo[movieInfo['영화명'] == movieName].loc[:, '개봉일'])[0]
            year, month, day = publishedAfter.split('-')
            publishedBefore = (datetime.datetime(int(year), int(month), int(day)) + datetime.timedelta(days=30)).strftime('%Y-%m-%d')
            q = ' '.join([movieName, keyword])

            search_params['q'] = q
            search_params['publishedAfter'] = publishedAfter+'T00:00:00Z'
            search_params['publishedBefore'] = publishedBefore+'T00:00:00Z'

            res_search = requests.get(search_base_url, search_params).json()

            # 네트워크 오류 발생 시
            while 'error' in res_search.keys():
                print('[ERROR] search requests ', res_search)
                tokenNum += 1
                if tokenNum > tokenLimit: 
                    print('[Error] token 오늘 할당량 끝났다~~', movieName, keyword)
                    return video_list

                print('[HTTPError] token 교체')
                search_params['key'] = env.token[tokenNum]
                res_search = requests.get(search_base_url, search_params).json()

            # 응답 데이터 가공
            if 'items' in res_search.keys():
                for item in res_search['items']:
                    result_video_ids = list(video_list.loc[video_list['영화 제목'] == movieName,'영상 id'])

                    if item['id']['videoId'] not in result_video_ids:
                        videos_params['id'] = item['id']['videoId']
                        res_videos = requests.get(videos_base_url, videos_params).json()

                        # 네트워크 오류 발생 시
                        while 'error' in res_videos.keys():
                            print('[ERROR] video requests ', res_videos)
                            tokenNum += 1
                            if tokenNum > tokenLimit: 
                                print('[Error] token 오늘 할당량 끝났다~~', movieName)
                                return video_list

                            print('[HTTPError] token 교체')
                            videos_params['key'] = env.token[tokenNum]
                            res_videos = requests.get(videos_base_url, videos_params).json()

                        if 'items' in res_videos.keys():
                            try:
                                video_statistics = res_videos['items'][0]['statistics']
                                item_data = [[
                                    item['id']['videoId'], 
                                    movieName, 
                                    q, 
                                    item['snippet']['title'], 
                                    int(video_statistics['viewCount']) if 'viewCount' in video_statistics else 0,
                                    int(video_statistics['likeCount']) if 'likeCount' in video_statistics else 0,
                                    int(video_statistics['commentCount']) if 'commentCount' in video_statistics else 0,
                                    item['snippet']['publishedAt'], 
                                ]]
                                video_list = video_list.append(pd.DataFrame(item_data, columns=columns),ignore_index=True)
                            except Exception as e:
                                print('[ERROR] ', e, movieName, res_videos)
                                return video_list


    return video_list

In [17]:
video_list = get_video_list()

오징어 게임 0
킹덤 50
보건교사 안은영 86
옥자 129
승리호 151


In [18]:
result = video_list.set_index('영상 id')

In [19]:
result.to_csv('./videoList/movieVideoList_netflix.csv', encoding='utf-8-sig')

In [20]:
top = lambda x: x.sort_values(by='조회수', ascending=False)[:10]
filtered_result = result.groupby('영화 제목').apply(top)
filtered_result

영화 제목         검색 키워드  \
영화 제목    영상 id                                  
보건교사 안은영 v5xMK2SvVls  보건교사 안은영  보건교사 안은영 넷플릭스   
         FrZpeDf7TVk  보건교사 안은영  보건교사 안은영 넷플릭스   
         -XM_XXlQxTw  보건교사 안은영  보건교사 안은영 넷플릭스   
         oa2GIfnVbB8  보건교사 안은영  보건교사 안은영 넷플릭스   
         YehiRW7-QeQ  보건교사 안은영  보건교사 안은영 넷플릭스   
         CLhdLXqAgEE  보건교사 안은영  보건교사 안은영 넷플릭스   
         71MtlkCaM08  보건교사 안은영  보건교사 안은영 넷플릭스   
         GExtZBvcP1Q  보건교사 안은영  보건교사 안은영 넷플릭스   
         9uCsGDHGXk8  보건교사 안은영  보건교사 안은영 넷플릭스   
         vN31iyva_4s  보건교사 안은영  보건교사 안은영 넷플릭스   
승리호      9XlHw1_9eWc       승리호       승리호 넷플릭스   
         U3VcG3P5aMw       승리호       승리호 넷플릭스   
         NIMtsF8DCOM       승리호       승리호 넷플릭스   
         sm6wGZOs0O8       승리호       승리호 넷플릭스   
         X0eCdVGR190       승리호       승리호 넷플릭스   
         bf2MCTNUm78       승리호       승리호 넷플릭스   
         jm2t1eOi1b4       승리호       승리호 넷플릭스   
         u__9KKrOSdk       승리호       승리호 넷플릭스   
         KM52IPhWdGU       승리호       승리호 넷플릭스   
         3BbVvCPdmXc       승리호       승리호 넷플릭스   
오징어 게임   RtPb4NMkT-s    오징어 게임    오징어 게임 넷플릭스   
         iRBQGFN-PEY    오징어 게임    오징어 게임 넷플릭스   
         TYd_pT9hZrM    오징어 게임    오징어 게임 넷플릭스   
         SXr8Rb97nIk    오징어 게임    오징어 게임 넷플릭스   
         T5r4tbdZ3pg    오징어 게임     오징어 게임 명장면   
         9OZczRT14DA    오징어 게임    오징어 게임 넷플릭스   
         U872VWMv4C0    오징어 게임    오징어 게임 넷플릭스   
         Ao9LWJqtiIg    오징어 게임    오징어 게임 넷플릭스   
         o4EF1NG_xks    오징어 게임    오징어 게임 넷플릭스   
         PSGKK1YXg5g    오징어 게임    오징어 게임 넷플릭스   
옥자       FRINKV5zZDU        옥자        옥자 넷플릭스   
         eOdMVj1mnHE        옥자        옥자 넷플릭스   
         1Xw6UL4fCko        옥자        옥자 넷플릭스   
         rKEkSFppdrw        옥자        옥자 넷플릭스   
         IutZs1uWRuM        옥자        옥자 넷플릭스   
         VdwxyD9-vLc        옥자        옥자 넷플릭스   
         CP2NOnRzELc        옥자        옥자 넷플릭스   
         cCmMKT8OaWM        옥자        옥자 넷플릭스   
         yjh1NC8bBSg        옥자        옥자 넷플릭스   
         WYPmhK3f8j8        옥자          옥자 해석   
킹덤       HczZsoj6IGM        킹덤        킹덤 넷플릭스   
         mmrvbB8K1yU        킹덤        킹덤 넷플릭스   
         iiki76Fg7Vc        킹덤        킹덤 넷플릭스   
         2oY8791RL0I        킹덤        킹덤 넷플릭스   
         g9auo2xIKuU        킹덤        킹덤 넷플릭스   
         WQNxEsHk4Cg        킹덤        킹덤 넷플릭스   
         GXyXqTAYhJk        킹덤        킹덤 넷플릭스   
         7uz6T-DRjQk        킹덤        킹덤 넷플릭스   
         2SYu0RSgziw        킹덤        킹덤 넷플릭스   
         ozJ2IJ2pA2Y        킹덤        킹덤 넷플릭스   

                                                                  영상 제목  \
영화 제목    영상 id                                                            
보건교사 안은영 v5xMK2SvVls                                 보건교사 안은영 [총몇명 특별편]   
         FrZpeDf7TVk                                [보건교사 안은영 OST] 도망가자   
         -XM_XXlQxTw                                  넷플릭스는 사드세요.....제발   
         oa2GIfnVbB8  이상하기로 소문난 OST [보건교사 안은영]  | The Weirdest OST ever   
         YehiRW7-QeQ  [#뭅텔러] 버르장머리 없는 원어민 교사 참교육(?) 시키는 보건교사 | 보건교사 ...   
         CLhdLXqAgEE  [광고]보건교사 안은영 수제북 ASMR｜정유미, 남주혁 주연 _ 넷플릭스 #엉뚱#유...   
         71MtlkCaM08        [#뭅뭅픽] 보건교사 안은영 속 형형색색 젤리 모음.zip | 보건교사 안은영   
         GExtZBvcP1Q             학교에 1200톤 초대형 두선생이 나타났을 때, 상상도 못했던 퇴치법   
         9uCsGDHGXk8                                한국 고등학교에 나타난 초대형 괴수   
         vN31iyva_4s  📺🌈🧪🧬🦠🐛Mysterious Jelly Monster world❣🩺👩🏻‍⚕️🔫 ☠...   
승리호      9XlHw1_9eWc  (ENG) 유난히 내성적이었던 김태리와 승리호 친구들은 왜 우주로 갔을까 would...   
         U3VcG3P5aMw       Cast of Space Sweepers plays Jenga [ENG SUB]   
         NIMtsF8DCOM  현재 넷플릭스 승리호 전세계 1위 찍자 미국에서 대반전 반응 벌어진 이유, 중국, ...   
         sm6wGZOs0O8                        승리호, 닉값을... 못했습니다... 승리호 리뷰   
         X0eCdVGR190  송중기, 김태리, 진선규, 유해진이 직접 밝힌 도로시 썰🌟| 승리호 Space Sw...   
         bf2MCTNUm78                      우주를 청소하는 악동들 [승리호][영화리뷰/결말포함]   
         jm2t1eOi1b4  송중기, 김태리 국민 MC 등극! 진선규, 유해진 찐 TMI 대방출🔥| 승리호 | ...   
         u__9KKrOSdk                              한

In [21]:
filtered_result.to_csv('./filteredVideoList/movieVideoList_netflix.csv', encoding='utf-8-sig')